# SmallData.jl

This project is part of a thought experiment. If there was only one computer shared by the whole world which had unlimited memory and could never crash, what would our databases look like. I want to figure this out and then see how close I can get with a real computer.

What I have so far is a higher level interface over SQLite.jl which presents SQL tables as if they were strongly typed in memory `Vectors` of objects.

In [1]:
@require "." Table DB;

## API

First you create a database connection. Normally you would pass in a filename where the database should be stored on disk but in this case we are just creating an in memory instance

In [2]:
db = DB()

SQLite.DB(in-memory)

Next we declare a table. If the `DB` already has this table then it will use it rather than create a new one

In [3]:
ranges = Table{UnitRange{Int}}(db)

0x2 UnitRange{Int64} Table
| start | stop |
|--------------|


From here on out everything you do is done the same way you would if `ranges` was a `Vector`

In [4]:
push!(ranges, 1:3)

1x2 UnitRange{Int64} Table
| start | stop |
|--------------|
|     1 |    3 |


In [5]:
ranges[1]

1:3

And we can have multiple instances of the same table and they share their underlying storage. This means you can have several processes sharing the same data

In [6]:
push!(Table{UnitRange{Int}}(db), 6:7, 1:5, 21:22)

4x2 UnitRange{Int64} Table
| start | stop |
|--------------|
|     1 |    3 |
|     6 |    7 |
|     1 |    5 |
|    21 |   22 |


## Filtering tables



Again the API is just as it would be with a `Vector` but I should mention this is a lot more efficient than you would guess. You probably think it would iterate over all values. It doesn't. Instead it compiles the function to SQL so it can be run by SQLite.

In [7]:
filter(range -> range.start == 1, ranges)

2x2 UnitRange{Int64} Table WHERE start = 1
| start | stop |
|--------------|
|     1 |    3 |
|     1 |    5 |
